In [1]:
import os
import torch
import torch.optim as optim
from torchtext.vocab import FastText
from utils.preprocess import MINDIterator
from utils.utils import getVocab,getLoss,getLabel,constructBasicDict,run_eval,run_train
from models.NPA import NPAModel

## setting up the *NPA* model

### define paths and hyperparameters, load data

all these hyper parameters are fixed according to the paper [\[23\] Npa Neural news recommendation with personalized attention](https://dl.acm.org/doi/abs/10.1145/3292500.3330665)

- *mode*: data to read (*demo*/*small*/*large*)

- *batch_size*: size of each minibatch

- *title_size*: max word capacity of title

- *his_size*: max record capacity of click history

- *npratio*: number of negtive sampling

- *dropout_p*: probability of dropout layer

- *filter_num*: number of kernels in 1D CNN, which is also embedding dimension of news/user

- *embedding_dim*: word embedding dimension

- *user_dim*: user id embedding dimension

- *preference_dim*: user preference embedding dimension

In [12]:
hparams = {
    'mode':'demo',
    'batch_size':50,#100,
    'title_size':30,
    'his_size':50,   
    'npratio':4,     
    'dropout_p':0.2,
    'filter_num':400,
    'embedding_dim':300,
    'user_dim':50,
    'preference_dim':200,
    'metrics':'group_auc,ndcg@4,mean_mrr',
    'gpu':'cuda:0',
    'attrs': ['title']
}

# customize your path here

news_file_train = 'D:/Data/NR_data/dev/news_train.tsv'
news_file_test = 'D:/Data/NR_data/dev/news_test.tsv'
behavior_file_train = 'D:/Data/NR_data/dev/behaviors_train.tsv'
behavior_file_test = 'D:/Data/NR_data/dev/behaviors_test.tsv'
save_path = 'models/model_param/NPA_'+ hparams['mode'] +'.model'

# if user2id,word2id,news2id haven't been constructed
if not os.path.exists('data/vocab_{}_{}_{}.pkl'.format(hparams['mode'],'train','_'.join(hparams['attrs']))):
    constructBasicDict(news_file_train,behavior_file_train,hparams['mode'],'train',hparams['attrs'])

if not os.path.exists('data/vocab_{}_{}_{}.pkl'.format(hparams['mode'],'test','_'.join(hparams['attrs']))):
    constructBasicDict(news_file_test,behavior_file_test,hparams['mode'],'test',hparams['attrs'])

device = torch.device(hparams['gpu']) if torch.cuda.is_available() else torch.device("cpu")

iterator_train = MINDIterator(hparams=hparams,mode='train',news_file=news_file_train,behaviors_file=behavior_file_train)

iterator_test = MINDIterator(hparams=hparams,mode='test',news_file=news_file_test,behaviors_file=behavior_file_test)

vocab_train = iterator_train.vocab
embedding = FastText('simple',cache='.vector_cache')
vocab_train.load_vectors(embedding)

vocab_test = iterator_test.vocab
vocab_test.load_vectors(embedding)

In [17]:
# you can load my model or train yours
if os.path.exists(save_path):
    npaModel = NPAModel(vocab=vocab_train,hparams=hparams)
    npaModel.load_state_dict(torch.load(save_path))
    npaModel.to(device).eval()

else:
    npaModel = NPAModel(vocab=vocab_train,hparams=hparams).to(device)
    npaModel.train()

### train model

In [18]:
if npaModel.training:
    loss_func = getLoss(npaModel)
    optimizer = optim.Adam(npaModel.parameters(),lr=0.0002)

    npaModel = run_train(npaModel,iterator_train,optimizer,loss_func, epochs=5, interval=15)

epoch 0 , step 45 , total_loss: 1.6102, batch_loss: 1.5801: : 59it [00:22,  2.68it/s]
epoch 1 , step 45 , total_loss: 1.6171, batch_loss: 1.5487: : 59it [00:21,  2.80it/s]
epoch 2 , step 45 , total_loss: 1.6253, batch_loss: 1.6265: : 59it [00:21,  2.76it/s]
epoch 3 , step 45 , total_loss: 1.6224, batch_loss: 1.6114: : 59it [00:21,  2.77it/s]
epoch 4 , step 45 , total_loss: 1.6025, batch_loss: 1.5346: : 59it [00:20,  2.85it/s]


### test & evaluate

In [19]:
npaModel.eval()
npaModel.vocab = vocab_test
npaModel.npratio = -1
iterator_test.npratio = -1

run_eval(npaModel,iterator_test)

{'group_auc': 0.5229, 'ndcg@4': 0.2067, 'mean_mrr': 0.2206}

### save the model

In [20]:
npaModel.npratio = 4
torch.save(npaModel.state_dict(), save_path)